In [35]:
from Bio.KEGG import REST
from Bio.KEGG import Enzyme

import gzip
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
enzyme_fields = [method for method in dir(Enzyme.Record()) if not method.startswith('_')]
data_matrix = []

with gzip.open('../datasets/KEGG_enzymes_all_data.gz', 'rt') as file:
    for record in Enzyme.parse(file):
        data_matrix.append([getattr(record, field) for field in enzyme_fields])

In [6]:
enzyme_df = pd.DataFrame(data_matrix, columns=enzyme_fields)

In [7]:
enzyme_df.head()

classname cofactor  \
0  [Oxidoreductases;, Acting on the CH-OH group o...       []   
1  [Oxidoreductases;, Acting on the CH-OH group o...       []   
2  [Oxidoreductases;, Acting on the CH-OH group o...       []   
3  [Oxidoreductases;, Acting on the CH-OH group o...       []   
4  [Oxidoreductases;, Acting on the CH-OH group o...       []   

                                             comment  \
0  [A zinc protein. Acts on primary or secondary ...   
1  [A zinc protein. Some members of this group ox...   
2  [The yeast enzyme acts most rapidly with NAD+;...   
3  [Also converts diacetyl into acetoin with NADH...   
4  [Transferred entry: acetoin dehydrogenase. Now...   

                                             dblinks disease effector  \
0  [(ExplorEnz - The Enzyme Database, [1.1.1.1]),...      []       []   
1  [(ExplorEnz - The Enzyme Database, [1.1.1.2]),...      []       []   
2  [(ExplorEnz - The Enzyme Database, [1.1.1.3]),...      []       []   
3  [(ExplorEnz - The Enzyme Database, [1.1.1.4]),...      []       []   
4                                                 []      []       []   

     entry                                              genes inhibitor  \
0  1.1.1.1  [(HSA, [124, 125, 126, 127, 128, 130, 131]), (...        []   
1  1.1.1.2  [(HSA, [10327]), (PTR, [741418]), (PPS, [10099...        []   
2  1.1.1.3  [(NVE, [NEMVE_v1g225948]), (ATH, [AT1G31230, A...        []   
3  1.1.1.4  [(SCE, [YAL060W, YAL061W]), (KLA, [KLLA0_F0050...        []   
4  1.1.1.5                                                 []        []   

                                                name  \
0  [alcohol dehydrogenase, aldehyde reductase, AD...   
1  [alcohol dehydrogenase (NADP+), aldehyde reduc...   
2              [homoserine dehydrogenase, HSDH, HSD]   
3  [(R,R)-butanediol dehydrogenase, butyleneglyco...   
4           [Transferred to 1.1.1.303 and 1.1.1.304]   

                                             pathway  \
0  [(PATH, ec00010, Glycolysis / Gluconeogenesis)...   
1  [(PATH, ec00010, Glycolysis / Gluconeogenesis)...   
2  [(PATH, ec00260, Glycine, serine and threonine...   
3            [(PATH, ec00650, Butanoate metabolism)]   
4                                                 []   

                                             product  \
0  [aldehyde [CPD:C00071], NADH [CPD:C00004], H+ ...   
1  [aldehyde [CPD:C00071], NADPH [CPD:C00005], H+...   
2  [L-aspartate 4-semialdehyde [CPD:C00441], NADH...   
3  [(R)-acetoin [CPD:C00810], NADH [CPD:C00004], ...   
4                                                 []   

                                            reaction structures  \
0  [(1) a primary alcohol + NAD+ = an aldehyde + ...         []   
1  [an alcohol + NADP+ = an aldehyde + NADPH + H+...         []   
2  [L-homoserine + NAD(P)+ = L-aspartate 4-semial...         []   
3  [(R,R)-butane-2,3-diol + NAD+ = (R)-acetoin + ...         []   
4                                                 []         []   

                                           substrate  \
0  [primary alcohol [CPD:C00226], NAD+ [CPD:C0000...   
1         [alcohol [CPD:C00069], NADP+ [CPD:C00006]]   
2  [L-homoserine [CPD:C00263], NAD+ [CPD:C00003],...   
3  [(R,R)-butane-2,3-diol [CPD:C03044], NAD+ [CPD...   
4                                                 []   

                                       sysname  
0                [alcohol:NAD+ oxidoreductase]  
1               [alcohol:NADP+ oxidoreductase]  
2        [L-homoserine:NAD(P)+ oxidoreductase]  
3  [(R,R)-butane-2,3-diol:NAD+ oxidoreductase]  
4                                           []

In [8]:
# enzyme df search 
enzyme_df[enzyme_df.entry == '1.1.1.153']['reaction']

152    [(1) L-erythro-7,8-dihydrobiopterin + NADP+ = ...
Name: reaction, dtype: object

In [9]:
enzyme_df['reaction'][153]

['(S)-ureidoglycolate + NAD(P)+ = oxalureate + NAD(P)H + H+ [RN:R02935 R02936]']

In [10]:
promiscuous_df = enzyme_df[[True if len(rxn) > 1 else False for rxn in enzyme_df['reaction']]]
compact_promiscuous_df = promiscuous_df[['entry','reaction','product','substrate']]
compact_promiscuous_df.iloc[1,2]

['pyruvate [CPD:C00022]', 'CO2 [CPD:C00011]', 'NADH [CPD:C00004]']

In [ ]:

# create a list of reactions that appear in promiscuous enzyme dataframe 
reaction_list = []
for index,row in compact_promiscuous_df.iterrows():
     for reaction in row[1]:
            if reaction.split("[RN:")[-1].startswith("R"):
                if not reaction.split("[RN:")[-1].startswith("RN"):
                     for i in reaction.split("[RN:")[-1][:-1].split(" "):
                            reaction_list.append(i)
            
reaction_list       


In [ ]:
# run it when there is good internet connection
# append all the reactions that are reversible 
reversible_reaction = []
for reaction in reaction_list:
    reaction_file = REST.kegg_get(reaction).read()
    for i in reaction_file.rstrip().split("\n"):
        if i.startswith("EQUATION") and "<=>" in i:
            reversible_reaction.append(reaction)
            print (reaction)

In [127]:
# it seem like all the reactions are reversible 
len(reversible_reaction)

1302

In [11]:
rowindex = np.arange(0,len(compact_promiscuous_df))
rowindex

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [12]:
compact_promiscuous_df_index=compact_promiscuous_df.set_index(rowindex)
compact_promiscuous_df_index.iloc[1,2]


['pyruvate [CPD:C00022]', 'CO2 [CPD:C00011]', 'NADH [CPD:C00004]']

### append substrate molecules to product column

In [13]:
# do not run this cell several times! it will append substrate molecules multiple times 
for index,row in compact_promiscuous_df_index.iterrows():
    productlist = row['product']
    substratelist = row['substrate']
    for substrate in substratelist:
        productlist.append(substrate)
    compact_promiscuous_df_index.iloc[index,2] = productlist

In [14]:
substrate_to_product_promiscuous_df = compact_promiscuous_df_index
len(substrate_to_product_promiscuous_df.iloc[1,2])

6

In [15]:
substrate_to_product_promiscuous_df = substrate_to_product_promiscuous_df[['entry','reaction','product']]

In [153]:
# save substrate and product combined dataframe to csv 
# might remove this dataframe from the git repo soon 
# substrate_to_product_promiscuous_df.to_csv("../datasets/substrate_product_combined_promiscuous.csv")

### cofactor removal 

In [16]:
substrate_to_product_promiscuous_df 

entry                                           reaction  \
0      1.1.1.1  [(1) a primary alcohol + NAD+ = an aldehyde + ...   
1     1.1.1.38  [(1) (S)-malate + NAD+ = pyruvate + CO2 + NADH...   
2     1.1.1.40  [(1) (S)-malate + NADP+ = pyruvate + CO2 + NAD...   
3     1.1.1.42  [isocitrate + NADP+ = 2-oxoglutarate + CO2 + N...   
4     1.1.1.85  [(2R,3S)-3-isopropylmalate + NAD+ = 4-methyl-2...   
5    1.1.1.110  [(1) (R)-3-(phenyl)lactate + NAD+ = 3-phenylpy...   
6    1.1.1.153  [(1) L-erythro-7,8-dihydrobiopterin + NADP+ = ...   
7    1.1.1.187  [(1) GDP-alpha-D-rhamnose + NAD(P)+ = GDP-4-de...   
8    1.1.1.203  [(1) beta-D-galacturonate + NAD+ = D-galactaro...   
9    1.1.1.237  [(1) (R)-3-(4-hydroxyphenyl)lactate + NAD(P)+ ...   
10   1.1.1.276  [L-serine + NADP+ = 2-aminoacetaldehyde + CO2 ...   
11   1.1.1.282  [(1) L-quinate + NAD(P)+ = 3-dehydroquinate + ...   
12   1.1.1.286  [(1) isocitrate + NAD+ = 2-oxoglutarate + CO2 ...   
13   1.1.1.287  [(1) D-arabinitol + NADP+ = D-xylulose + NADPH...   
14   1.1.1.297  [menth-8-ene-1,2-diol + NAD+ = 1-hydroxymenth-...   
15   1.1.1.324  [(6E)-8-hydroxygeraniol + 2 NADP+ = (6E)-8-oxo...   
16   1.1.1.325  [(1) L-threo-7,8-dihydrobiopterin + NADP+ = se...   
17   1.1.1.352  [(1) (1'S,5'S)-hydroxyaverantin + NAD+ = 5'-ox...   
18   1.1.1.353  [(1) versicolorone + NADP+ = 1'-hydroxyversico...   
19   1.1.1.360  [(1) D-glucopyranose + NADP+ = D-glucono-1,5-l...   
20   1.1.1.372  [(1) glycerol + NADP+ = L-glyceraldehyde + NAD...   
21   1.1.1.381  [L-allo-threonine + NADP+ = aminoacetone + CO2...   
22   1.1.1.387  [L-serine + NAD+ = 2-aminoacetaldehyde + CO2 +...   
23   1.1.1.394  [aurachin B + NAD+ + H2O = 4-[(2E,6E)-farnesyl...   
24   1.1.1.396  [(1) 3-deacetyl-3-(1-hydroxyethyl)bacteriochlo...   
25   1.1.1.398  [2-(glutathion-S-yl)-2-methylbut-3-en-1-ol + 2...   
26   1.1.1.408  [4-phospho-D-threonate + NAD+ = glycerone phos...   
27   1.1.1.409  [4-phospho-D-erythronate + NAD+ = glycerone ph...   
28     1.1.3.8  [L-gulono-1,4-lactone + O2 = L-ascorbate + H2O...   
29    1.1.3.17  [choline + 2 O2 + H2O = betaine + 2 H2O2 (over...   
..         ...                                                ...   
519   5.5.1.24  [(1) delta-tocopherol = 2-methyl-6-phytylbenze...   
520   5.5.1.27  [(1) D-galactaro-1,4-lactone = 5-dehydro-4-deo...   
521    6.1.2.2  [(1) tobramycin + carbamoyl phosphate + ATP + ...   
522    6.2.1.7  [(1) ATP + cholate + CoA = AMP + diphosphate +...   
523   6.2.1.39  [(1) ATP + L-glutamate + BtrI acyl-carrier pro...   
524   6.2.1.49  [ATP + a long-chain fatty acid + holo-[mycocer...   
525   6.2.1.50  [ATP + 4-hydroxybenzoate + holo-[4-hydroxyphen...   
526   6.2.1.51  [(1) ATP + 17-(4-hydroxyphenyl)heptadecanoate ...   
527   6.2.1.53  [ATP + L-proline + holo-[L-prolyl-carrier prot...   
528   6.2.1.54  [ATP + D-alanine + holo-[D-alanyl-carrier prot...   
529   6.2.1.55  [ATP + [SAMP]-Gly-Gly + [E1 SAMP-activating en...   
530    6.3.1.9  [(1) glutathione + spermidine + ATP = glutathi...   
531   6.3.1.10  [(1) ATP + adenosylcobyric acid + (R)-1-aminop...   
532   6.3.1.20  [ATP + (R)-lipoate + a [lipoyl-carrier protein...   
533   6.3.2.47  [(1) ATP + 3-{[(2E)-4-amino-4-oxobut-2-enoyl]a...   
534    6.3.4.2  [ATP + UTP + L-glutamine = ADP + phosphate + C...   
535   6.3.4.16  [2 ATP + NH3 + hydrogencarbonate = 2 ADP + pho...   
536    6.3.5.2  [ATP + XMP + L-glutamine + H2O = AMP + diphosp...   
537    6.3.5.4  [ATP + L-aspartate + L-glutamine + H2O = AMP +...   
538    6.3.5.5  [2 ATP + L-glutamine + hydrogencarbonate + H2O...   
539   6.3.5.11  [2 ATP + cobyrinate + 2 L-glutamine + 2 H2O = ...   
540    6.5.1.1  [ATP + (deoxyribonucleotide)n-3'-hydroxyl + 5'...   
541    6.5.1.2  [NAD + (deoxyribonucleotide)n-3'-hydroxyl + 5'...   
542    6.5.1.3  [ATP + (ribonucleotide)n-3'-hydroxyl + 5'-phos...   
543    6.5.1.4  [ATP + [RNA]-3'-(3'-phospho-ribonucleoside) = ...   
544    6.5.1.5  [GTP + [RNA]-3'-(3'-phospho-ribonucleoside) = ...   
545    

In [17]:
compact_substrate_to_product_promiscuous_df = substrate_to_product_promiscuous_df[['entry','product']]

In [18]:
len(compact_substrate_to_product_promiscuous_df)

549

In [19]:
# test cleaning 
test='aldehyde [CPD:C00071]'
test[-7:-1]

'C00071'

In [20]:
#cofactor removal 
cofactor_df = pd.read_csv("cofactor_list.csv")
cofactor_list = [cofactor[4:10] for cofactor in cofactor_df.CPD]
cofactor_list
    

['C00001',
 'C00002',
 'C00003',
 'C00004',
 'C00005',
 'C00006',
 'C00007',
 'C00008',
 'C00009',
 'C00010',
 'C00011',
 'C00012',
 'C00013',
 'C00014',
 'C00015',
 'C00016',
 'C00017',
 'C00018',
 'C00019',
 'C00020',
 'C00021',
 'C00023',
 'C00027',
 'C00028',
 'C00030',
 'C00032',
 'C00034',
 'C00050',
 'C00061',
 'C00070',
 'C00080',
 'C00255',
 'C01007',
 'C01352',
 'C01382',
 'C02745',
 'C02869']

In [21]:
newcompoundcolumn=[]
newdf = compact_substrate_to_product_promiscuous_df[['entry']]
newdf

            

entry
0      1.1.1.1
1     1.1.1.38
2     1.1.1.40
3     1.1.1.42
4     1.1.1.85
5    1.1.1.110
6    1.1.1.153
7    1.1.1.187
8    1.1.1.203
9    1.1.1.237
10   1.1.1.276
11   1.1.1.282
12   1.1.1.286
13   1.1.1.287
14   1.1.1.297
15   1.1.1.324
16   1.1.1.325
17   1.1.1.352
18   1.1.1.353
19   1.1.1.360
20   1.1.1.372
21   1.1.1.381
22   1.1.1.387
23   1.1.1.394
24   1.1.1.396
25   1.1.1.398
26   1.1.1.408
27   1.1.1.409
28     1.1.3.8
29    1.1.3.17
..         ...
519   5.5.1.24
520   5.5.1.27
521    6.1.2.2
522    6.2.1.7
523   6.2.1.39
524   6.2.1.49
525   6.2.1.50
526   6.2.1.51
527   6.2.1.53
528   6.2.1.54
529   6.2.1.55
530    6.3.1.9
531   6.3.1.10
532   6.3.1.20
533   6.3.2.47
534    6.3.4.2
535   6.3.4.16
536    6.3.5.2
537    6.3.5.4
538    6.3.5.5
539   6.3.5.11
540    6.5.1.1
541    6.5.1.2
542    6.5.1.3
543    6.5.1.4
544    6.5.1.5
545    6.5.1.6
546    6.5.1.7
547    6.5.1.8
548    7.1.1.7

[549 rows x 1 columns]

In [22]:
# I wanted to use dataframe.loc but I was keep getting an error and this was the best way possible.. 
no_noncofactorcompound = []
compoundcolumn = []
for index,row in compact_substrate_to_product_promiscuous_df.iterrows():
    newcompoundlist = []
    for compound in row[1]:
        if "CPD" in compound:
            onlycpd = compound[-7:-1]
            if onlycpd not in cofactor_list:
                newcompoundlist.append(onlycpd)
    if len(newcompoundlist)==0:
        no_noncofactorcompound.append(row[0])
        compoundcolumn.append("NA")
    else:
        compoundcolumn.append(newcompoundlist)
newdf['product'] = compoundcolumn

        

In [23]:
#renamed
cleaned_promiscuous_enzyme_df=newdf

In [24]:
#no cofactor. #cleaned version 
cleaned_promiscuous_enzyme_df

entry                                            product
0      1.1.1.1                   [C00071, C01450, C00226, C01612]
1     1.1.1.38                           [C00022, C00149, C00036]
2     1.1.1.40                           [C00022, C00149, C00036]
3     1.1.1.42                   [C00026, C05379, C00311, C05379]
4     1.1.1.85                   [C00233, C04236, C04411, C04236]
5    1.1.1.110           [C01179, C00331, C05607, C03964, C22006]
6    1.1.1.153                   [C00835, C03684, C02953, C00272]
7    1.1.1.187                           [C01222, C03117, C02977]
8    1.1.1.203                                   [C20889, C20890]
9    1.1.1.237                   [C01179, C04045, C03964, C22038]
10   1.1.1.276                   [C06735, C11822, C00065, C11822]
11   1.1.1.282                   [C00944, C02637, C00296, C00493]
12   1.1.1.286                   [C00026, C00322, C00311, C05662]
13   1.1.1.287                           [C00310, C00309, C01904]
14   1.1.1.297           [C18019, C11937, C18020, C07276, C19082]
15   1.1.1.324   [C17622, C20222, C20223, C17621, C20222, C20223]
16   1.1.1.325                   [C00835, C03684, C20263, C20264]
17   1.1.1.352                           [C20502, C20500, C20501]
18   1.1.1.353   [C20503, C20505, C20507, C20504, C20506, C20508]
19   1.1.1.360                   [C00198, C02669, C00031, C00124]
20   1.1.1.372                           [C02426, C00577, C00116]
21   1.1.1.381                   [C01888, C03508, C05519, C03508]
22   1.1.1.387                   [C06735, C11822, C00065, C11822]
23   1.1.1.394                                           [C21140]
24   1.1.1.396                           [C18155, C18153, C18154]
25   1.1.1.398                                           [C21374]
26   1.1.1.408                   [C00111, C21610, C21585, C21610]
27   1.1.1.409                                   [C00111, C03393]
28     1.1.3.8                   [C00072, C03289, C01040, C03289]
29    1.1.3.17                   [C00719, C00576, C00114, C00576]
..         ...                                                ...
519   5.5.1.24  [C15882, C15883, C20737, C20738, C14151, C0248...
520   5.5.1.27                           [C00679, C20896, C21095]
521    6.1.2.2  [C18001, C20684, C20699, C00397, C00169, C2068...
522    6.2.1.7                                   [C01794, C00695]
523   6.2.1.39                                           [C00025]
524   6.2.1.49                                           [C00638]
525   6.2.1.50                           [C21497, C00156, C21497]
526   6.2.1.51                                   [C21446, C21448]
527   6.2.1.53                           [C20656, C00148, C20656]
528   6.2.1.54                                           [C00133]
529   6.2.1.55                           [C21901, C21900, C21901]
530    6.3.1.9           [C05730, C02090, C00051, C00315, C05730]
531   6.3.1.10           [C06509, C06508, C06507, C04122, C03194]
532   6.3.1.20           [C16237, C16238, C16241, C16240, C16238]
533   6.3.2.47  [C20962, C20963, C20964, C20965, C20966, C0018...
534    6.3.4.2                   [C00063, C00025, C00075, C00064]
535   6.3.4.16   [C00169, C20969, C01563, C00288, C20969, C01563]
536    6.3.5.2                   [C00144, C00025, C00655, C00064]
537    6.3.5.4                   [C00152, C00025, C00049, C00064]
538    6.3.5.5  [C00025, C00169, C20969, C01563, C00064, C0028...
539   6.3.5.11   [C06504, C00025, C19724, C05773, C00064, C19724]
540    6.5.1.1                                           [C00039]
541    6.5.1.2                                           [C00039]
542    6.5.1.3                                           [C00046]
543    6.5.1.4                                                 NA
544    6.5.1.5                                   [C00144, C00044]
545    6.5.1.6                                           [C00039]
546    6.5.1.7                           [C00039, C00144, C00044]
547    6.5.1.8                           [C00046,

In [25]:
#list of enzyme entries without compound, other than cofactors 
no_noncofactorcompound

['1.3.1.88',
 '1.3.1.90',
 '1.11.1.21',
 '1.14.13.107',
 '1.18.1.3',
 '2.1.1.200',
 '2.1.1.202',
 '2.1.1.207',
 '2.1.1.213',
 '2.1.1.225',
 '2.1.1.244',
 '2.1.1.268',
 '2.3.2.24',
 '2.3.2.25',
 '2.3.2.26',
 '2.3.2.31',
 '3.1.11.7',
 '4.1.1.68',
 '4.2.1.161',
 '4.2.1.166',
 '4.2.1.170',
 '4.4.1.29',
 '4.4.1.30',
 '4.6.1.18',
 '4.6.1.20',
 '4.6.1.21',
 '5.3.3.8',
 '6.5.1.4']

In [26]:
#cleaned_promiscuous_enzyme_df.to_csv("../datasets/cleaned_promiscous_enzyme_df.csv", header=['entry','product'])

In [66]:
noNAenzyme = cleaned_promiscuous_enzyme_df.loc[cleaned_promiscuous_enzyme_df['product']!='NA']

In [68]:
noNAenzyme = noNAenzyme.rename(columns={'product':'products'})
noNAenzyme

entry                                           products
0      1.1.1.1                   [C00071, C01450, C00226, C01612]
1     1.1.1.38                           [C00022, C00149, C00036]
2     1.1.1.40                           [C00022, C00149, C00036]
3     1.1.1.42                   [C00026, C05379, C00311, C05379]
4     1.1.1.85                   [C00233, C04236, C04411, C04236]
5    1.1.1.110           [C01179, C00331, C05607, C03964, C22006]
6    1.1.1.153                   [C00835, C03684, C02953, C00272]
7    1.1.1.187                           [C01222, C03117, C02977]
8    1.1.1.203                                   [C20889, C20890]
9    1.1.1.237                   [C01179, C04045, C03964, C22038]
10   1.1.1.276                   [C06735, C11822, C00065, C11822]
11   1.1.1.282                   [C00944, C02637, C00296, C00493]
12   1.1.1.286                   [C00026, C00322, C00311, C05662]
13   1.1.1.287                           [C00310, C00309, C01904]
14   1.1.1.297           [C18019, C11937, C18020, C07276, C19082]
15   1.1.1.324   [C17622, C20222, C20223, C17621, C20222, C20223]
16   1.1.1.325                   [C00835, C03684, C20263, C20264]
17   1.1.1.352                           [C20502, C20500, C20501]
18   1.1.1.353   [C20503, C20505, C20507, C20504, C20506, C20508]
19   1.1.1.360                   [C00198, C02669, C00031, C00124]
20   1.1.1.372                           [C02426, C00577, C00116]
21   1.1.1.381                   [C01888, C03508, C05519, C03508]
22   1.1.1.387                   [C06735, C11822, C00065, C11822]
23   1.1.1.394                                           [C21140]
24   1.1.1.396                           [C18155, C18153, C18154]
25   1.1.1.398                                           [C21374]
26   1.1.1.408                   [C00111, C21610, C21585, C21610]
27   1.1.1.409                                   [C00111, C03393]
28     1.1.3.8                   [C00072, C03289, C01040, C03289]
29    1.1.3.17                   [C00719, C00576, C00114, C00576]
..         ...                                                ...
518    5.4.3.3                   [C01186, C05161, C01142, C00739]
519   5.5.1.24  [C15882, C15883, C20737, C20738, C14151, C0248...
520   5.5.1.27                           [C00679, C20896, C21095]
521    6.1.2.2  [C18001, C20684, C20699, C00397, C00169, C2068...
522    6.2.1.7                                   [C01794, C00695]
523   6.2.1.39                                           [C00025]
524   6.2.1.49                                           [C00638]
525   6.2.1.50                           [C21497, C00156, C21497]
526   6.2.1.51                                   [C21446, C21448]
527   6.2.1.53                           [C20656, C00148, C20656]
528   6.2.1.54                                           [C00133]
529   6.2.1.55                           [C21901, C21900, C21901]
530    6.3.1.9           [C05730, C02090, C00051, C00315, C05730]
531   6.3.1.10           [C06509, C06508, C06507, C04122, C03194]
532   6.3.1.20           [C16237, C16238, C16241, C16240, C16238]
533   6.3.2.47  [C20962, C20963, C20964, C20965, C20966, C0018...
534    6.3.4.2                   [C00063, C00025, C00075, C00064]
535   6.3.4.16   [C00169, C20969, C01563, C00288, C20969, C01563]
536    6.3.5.2                   [C00144, C00025, C00655, C00064]
537    6.3.5.4                   [C00152, C00025, C00049, C00064]
538    6.3.5.5  [C00025, C00169, C20969, C01563, C00064, C0028...
539   6.3.5.11   [C06504, C00025, C19724, C05773, C00064, C19724]
540    6.5.1.1                                           [C00039]
541    6.5.1.2                                           [C00039]
542    6.5.1.3                                           [C00046]
544    6.5.1.5                                   [C00144, C00044]
545    6.5.1.6                                           [C00039]
546    6.5.1.7                           [C00039, C00144, C00044]
547    6.5.1.8                           [C00046,

In [80]:
noNAenzyme_expand = noNAenzyme.products.apply(pd.Series).merge(noNAenzyme, left_index = True, right_index = True).drop(["products"], axis = 1).melt(id_vars = ['entry'], value_name = "product")


In [117]:
noNAenzyme_expand.sort_values(by=['entry'],inplace=True)

In [118]:
new = noNAenzyme_expand.dropna().drop(columns=['variable']).drop_duplicates(['entry','product'],keep='first')
compoundList = list(new['product'])

In [119]:
compoundList




['C00071',
 'C01612',
 'C01450',
 'C00226',
 'C05607',
 'C22006',
 'C01179',
 'C00331',
 'C03964',
 'C02953',
 'C00272',
 'C03684',
 'C00835',
 'C01222',
 'C03117',
 'C02977',
 'C20890',
 'C20889',
 'C01179',
 'C03964',
 'C22038',
 'C04045',
 'C00065',
 'C06735',
 'C11822',
 'C00944',
 'C02637',
 'C00493',
 'C00296',
 'C00311',
 'C05662',
 'C00322',
 'C00026',
 'C00310',
 'C00309',
 'C01904',
 'C07276',
 'C18020',
 'C11937',
 'C18019',
 'C19082',
 'C20223',
 'C20222',
 'C17621',
 'C17622',
 'C20263',
 'C03684',
 'C20264',
 'C00835',
 'C20501',
 'C20502',
 'C20500',
 'C20506',
 'C20503',
 'C20507',
 'C20508',
 'C20505',
 'C20504',
 'C02669',
 'C00198',
 'C00124',
 'C00031',
 'C00116',
 'C00577',
 'C02426',
 'C00149',
 'C00036',
 'C00022',
 'C05519',
 'C03508',
 'C01888',
 'C06735',
 'C11822',
 'C00065',
 'C21140',
 'C18155',
 'C18154',
 'C18153',
 'C21374',
 'C00149',
 'C00022',
 'C00036',
 'C21610',
 'C00111',
 'C21585',
 'C03393',
 'C00111',
 'C00311',
 'C05379',
 'C00026',
 'C04236',

In [120]:
new

entry product
0       1.1.1.1  C00071
1563    1.1.1.1  C01612
521     1.1.1.1  C01450
1042    1.1.1.1  C00226
1047  1.1.1.110  C05607
2089  1.1.1.110  C22006
5     1.1.1.110  C01179
526   1.1.1.110  C00331
1568  1.1.1.110  C03964
1048  1.1.1.153  C02953
1569  1.1.1.153  C00272
527   1.1.1.153  C03684
6     1.1.1.153  C00835
7     1.1.1.187  C01222
528   1.1.1.187  C03117
1049  1.1.1.187  C02977
529   1.1.1.203  C20890
8     1.1.1.203  C20889
9     1.1.1.237  C01179
1051  1.1.1.237  C03964
1572  1.1.1.237  C22038
530   1.1.1.237  C04045
1052  1.1.1.276  C00065
10    1.1.1.276  C06735
531   1.1.1.276  C11822
11    1.1.1.282  C00944
532   1.1.1.282  C02637
1574  1.1.1.282  C00493
1053  1.1.1.282  C00296
1054  1.1.1.286  C00311
...         ...     ...
2075   6.3.5.11  C05773
2072    6.3.5.2  C00064
1551    6.3.5.2  C00655
1030    6.3.5.2  C00025
509     6.3.5.2  C00144
2073    6.3.5.4  C00064
1031    6.3.5.4  C00025
510     6.3.5.4  C00152
1552    6.3.5.4  C00049
2595    6.3.5.5  C00064
3637    6.3.5.5  C20969
4158    6.3.5.5  C01563
1032    6.3.5.5  C00169
511     6.3.5.5  C00025
3116    6.3.5.5  C00288
513     6.5.1.1  C00039
514     6.5.1.2  C00039
515     6.5.1.3  C00046
516     6.5.1.5  C00144
1037    6.5.1.5  C00044
517     6.5.1.6  C00039
1039    6.5.1.7  C00144
518     6.5.1.7  C00039
1560    6.5.1.7  C00044
519     6.5.1.8  C00046
1040    6.5.1.8  C00144
1561    6.5.1.8  C00044
520     7.1.1.7  C00399
1041    7.1.1.7  C05359
1562    7.1.1.7  C00390

[2144 rows x 2 columns]

In [121]:
import re
from Bio.KEGG import Compound

In [122]:
def compound_records_to_df(file_path):
    """
    Input should be a filepath string pointing to a gzipped text file of KEGG enzyme records.
    Function parses all records using Biopython.Bio.KEGG.Compound parser, and returns a pandas dataframe.
    """
    compound_fields = [method for method in dir(Compound.Record()) if not method.startswith('_')]
    data_matrix = []

    with gzip.open(file_path, 'rt') as file:
        for record in Compound.parse(file):
            data_matrix.append([getattr(record, field) for field in compound_fields])
    
    compound_df = pd.DataFrame(data_matrix, columns=compound_fields)
    return compound_df

In [123]:
compound_df = compound_records_to_df('../datasets/KEGG_compound_db_entries.gz')

In [124]:
def extract_PubChem_id(field):
    """
    This function uses regular expressions to extract the PubChem compound IDs from a field in a record
    """

    regex = "'PubChem', \[\'(\d+)\'\]\)" # matches "'PubChem', ['" characters exactly, then captures any number of digits (\d+), before another literal "']" character match
    ids = re.findall(regex, str(field), re.IGNORECASE)
    if len(ids) > 0:
        pubchem_id = ids[0]
    else:
        pubchem_id = ''
    
    return pubchem_id

In [125]:
PubChemID_list = []

for _, row in compound_df.iterrows():
    pubchem_id = extract_PubChem_id(row['dblinks'])
    PubChemID_list.append(pubchem_id)
    
compound_df['PubChem'] = PubChemID_list
compound_df.head(10)

dblinks   entry  \
0  [(CAS, [7732-18-5]), (PubChem, [3303]), (ChEBI...  C00001   
1  [(CAS, [56-65-5]), (PubChem, [3304]), (ChEBI, ...  C00002   
2  [(CAS, [53-84-9]), (PubChem, [3305]), (ChEBI, ...  C00003   
3  [(CAS, [58-68-4]), (PubChem, [3306]), (ChEBI, ...  C00004   
4  [(CAS, [2646-71-1]), (PubChem, [3307]), (ChEBI...  C00005   
5  [(CAS, [53-59-8]), (PubChem, [3308]), (ChEBI, ...  C00006   
6  [(CAS, [7782-44-7]), (PubChem, [3309]), (ChEBI...  C00007   
7  [(CAS, [58-64-0]), (PubChem, [3310]), (ChEBI, ...  C00008   
8  [(CAS, [7664-38-2]), (PubChem, [3311]), (ChEBI...  C00009   
9  [(CAS, [85-61-0]), (PubChem, [3312]), (ChEBI, ...  C00010   

                                              enzyme         formula mass  \
0  [1.1.1.1, 1.1.1.22, 1.1.1.23, 1.1.1.115, 1.1.1...             H2O        
1  [1.1.98.6, 1.2.1.30, 1.2.1.95, 1.2.1.101, 1.3....   C10H16N5O13P3        
2  [1.1.1.1, 1.1.1.3, 1.1.1.4, 1.1.1.6, 1.1.1.7, ...   C21H28N7O14P2        
3  [1.1.1.1, 1.1.1.3, 1.1.1.4, 1.1.1.6, 1.1.1.7, ...   C21H29N7O14P2        
4  [1.1.1.1, 1.1.1.2, 1.1.1.3, 1.1.1.10, 1.1.1.19...   C21H30N7O17P3        
5  [1.1.1.1, 1.1.1.2, 1.1.1.3, 1.1.1.10, 1.1.1.19...   C21H29N7O17P3        
6  [1.1.1.170, 1.1.1.270, 1.1.3.2, 1.1.3.4, 1.1.3...              O2        
7  [1.3.7.7, 1.3.7.8, 1.3.7.14, 1.3.7.15, 1.17.4....   C10H15N5O10P2        
8  [1.2.1.11, 1.2.1.12, 1.2.1.13, 1.2.1.38, 1.2.1...           H3PO4        
9  [1.1.1.34, 1.1.1.88, 1.1.1.-, 1.2.1.10, 1.2.1....  C21H36N7O16P3S        

                                                name  \
0                                       [H2O, Water]   
1                   [ATP, Adenosine 5'-triphosphate]   
2  [NAD+, NAD, Nicotinamide adenine dinucleotide,...   
3  [NADH, DPNH, Reduced nicotinamide adenine dinu...   
4  [NADPH, TPNH, Reduced nicotinamide adenine din...   
5  [NADP+, NADP, Nicotinamide adenine dinucleotid...   
6                                       [Oxygen, O2]   
7                    [ADP, Adenosine 5'-diphosphate]   
8  [Orthophosphate, Phosphate, Phosphoric acid, O...   
9                          [CoA, Coenzyme A, CoA-SH]   

                                             pathway structures PubChem  
0  [(PATH, map00190, Oxidative phosphorylation), ...         []    3303  
1  [(PATH, map00190, Oxidative phosphorylation), ...         []    3304  
2  [(PATH, map00190, Oxidative phosphorylation), ...         []    3305  
3  [(PATH, map00190, Oxidative phosphorylation), ...         []    3306  
4  [(PATH, map00195, Photosynthesis), (PATH, map0...         []    3307  
5  [(PATH, map00195, Photosynthesis), (PATH, map0...         []    3308  
6  [(PATH, map00190, Oxidative phosphorylation), ...         []    3309  
7  [(PATH, map00190, Oxidative phosphorylation), ...         []    3310  
8  [(PATH, map00190, Oxidative phosphorylation), ...         []    3311  
9  [(PATH, map00071, Fatty acid degradation), (PA...         []    3312

In [126]:
joint_enzyme_compound_df = new.merge(compound_df, left_on='product', right_on='entry')

In [127]:
joint_enzyme_compound_df=joint_enzyme_compound_df[['entry_x','product','formula','PubChem']].sort_values(by=['entry_x'])

In [128]:
joint_enzyme_compound_df.rename(columns={'entry_x':'entry','product':'KEGG'},inplace=True)

In [129]:
joint_enzyme_compound_df

entry    KEGG                   formula    PubChem
0       1.1.1.1  C00071                      CHOR       3371
2       1.1.1.1  C01612                    CH2OR2       4764
3       1.1.1.1  C01450                      COR2       4627
4       1.1.1.1  C00226                     CH3OR       3526
5     1.1.1.110  C05607                   C9H10O3       7930
6     1.1.1.110  C22006                 C11H11NO3           
7     1.1.1.110  C01179                    C9H8O4       4406
9     1.1.1.110  C00331                  C11H9NO3       3625
10    1.1.1.110  C03964                   C9H10O4       6685
16    1.1.1.153  C00835                 C9H11N5O3       4093
14    1.1.1.153  C03684                 C9H11N5O3       6459
12    1.1.1.153  C02953                 C9H13N5O3       5871
13    1.1.1.153  C00272                 C9H15N5O3       3570
18    1.1.1.187  C01222             C16H23N5O15P2       4444
20    1.1.1.187  C03117             C16H25N5O15P2       6011
21    1.1.1.187  C02977             C16H25N5O15P2       5888
22    1.1.1.203  C20890                    C6H8O7  254741353
23    1.1.1.203  C20889                    C6H8O7  254741352
8     1.1.1.237  C01179                    C9H8O4       4406
24    1.1.1.237  C22038                   C9H10O5           
25    1.1.1.237  C04045                    C9H8O5       6746
11    1.1.1.237  C03964                   C9H10O4       6685
26    1.1.1.276  C00065                   C3H7NO3       3365
31    1.1.1.276  C06735                    C2H5NO       8957
33    1.1.1.276  C11822                   C3H5NO3      13986
36    1.1.1.282  C02637                    C7H8O5       5617
37    1.1.1.282  C00493                   C7H10O5       3776
38    1.1.1.282  C00296                   C7H12O6       3590
35    1.1.1.282  C00944                   C7H10O6       4196
42    1.1.1.286  C05662                   C7H10O7       7972
...         ...     ...                       ...        ...
1154   6.3.5.11  C00025                   C5H9NO4       3327
1721    6.3.5.2  C00144               C10H14N5O8P       3444
1155    6.3.5.2  C00025                   C5H9NO4       3327
1166    6.3.5.2  C00064                 C5H10N2O3       3364
1948    6.3.5.2  C00655               C10H13N4O9P       3925
1156    6.3.5.4  C00025                   C5H9NO4       3327
1874    6.3.5.4  C00049                   C4H7NO4       3351
1871    6.3.5.4  C00152                  C4H8N2O3       3452
1167    6.3.5.4  C00064                 C5H10N2O3       3364
2137    6.3.5.5  C20969                    CH3O6P  254741428
1692    6.3.5.5  C00288                      HCO3       3583
1687    6.3.5.5  C01563                    CH3NO2       4721
1157    6.3.5.5  C00025                   C5H9NO4       3327
1682    6.3.5.5  C00169                   CH4NO5P       3469
1168    6.3.5.5  C00064                 C5H10N2O3       3364
1376    6.5.1.1  C00039    C10H17O8PR2(C5H8O5PR)n       3341
1377    6.5.1.2  C00039    C10H17O8PR2(C5H8O5PR)n       3341
2072    6.5.1.3  C00046  C10H18O13P2R2(C5H8O6PR)n       3348
1716    6.5.1.5  C00044             C10H16N5O14P3       3346
1722    6.5.1.5  C00144               C10H14N5O8P       3444
1378    6.5.1.6  C00039    C10H17O8PR2(C5H8O5PR)n       3341
1723    6.5.1.7  C00144               C10H14N5O8P       3444
1717    6.5.1.7  C00044             C10H16N5O14P3       3346
1379    6.5.1.7  C00039    C10H17O8PR2(C5H8O5PR)n       3341
1718    6.5.1.8  C00044             C10H16N5O14P3       3346
2073    6.5.1.8  C00046  C10H18O13P2R2(C5H8O6PR)n       3348
1724    6.5.1.8  C00144               C10H14N5O8P       3444
2142    7.1.1.7  C05359                                 7736
2141    7.1.1.7  C00399           C14H18O4(C5H8)n       3689
2143    7.1.1.7  C00390           C14H20O4(C5H8)n       3680

[2144 rows x 4 columns]